In [1]:
import sys
sys.path.append('../')
from graph import *
import pandas as pd

In [2]:
# Load unit path data
unit_path = pd.read_csv('../unit_path.csv')

In [3]:
G = base_graph_generation(unit_path)

In [4]:
source_node = 'schema.MusicAlbum'
target_node = 'schema.Country'

In [5]:
result = find_shortest_path(G, source_node, target_node)
result

[('schema.MusicAlbum', 'SKMO_v2.4_0223.hasTrack', 'mo.Track'),
 ('mo.Track', 'SKMO_v2.4_0223.isSungBy', 'mo.MusicGroup'),
 ('mo.MusicGroup', 'SKMO_v2.4_0223.debutNation', 'schema.Country')]

In [6]:
result = find_shortest_path(G, source_node, target=target_node, p_name='schema.nationality')
result

[('schema.MusicAlbum', 'SKMO_v2.4_0223.hasTrack', 'mo.Track'),
 ('mo.Track', 'SKMO_v2.4_0223.isSungBy', 'mo.MusicGroup'),
 ('mo.MusicGroup', 'SKMO_v2.4_0223.hasLeader', 'foaf.Person'),
 ('foaf.Person', 'schema.nationality', 'schema.Country')]

In [7]:
result = find_shortest_path(G, source_node, target=target_node, p_name='schema.nationality', n_triple=3)
result

[('schema.MusicAlbum', 'SKMO_v2.4_0223.hasSinger', 'foaf.Person'),
 ('foaf.Person', 'schema.nationality', 'schema.Country')]

In [8]:
result = find_shortest_path(G, source_node, p_name='schema.nationality')
result

[('schema.MusicAlbum', 'SKMO_v2.4_0223.hasTrack', 'mo.Track'),
 ('mo.Track', 'SKMO_v2.4_0223.isSungBy', 'mo.MusicGroup'),
 ('mo.MusicGroup', 'SKMO_v2.4_0223.hasLeader', 'foaf.Person'),
 ('foaf.Person', 'schema.nationality', 'schema.Country')]